In [92]:
from pulp import *
import pandas as pd

In [147]:
aa=pd.read_csv('./zscores.csv', index_col=0)

In [148]:
aa['Name'] = aa['FirstName'] + ' ' + aa['LastName']
aa.sort_values('Z_Sum', ascending=False)
aa.loc[aa['ProjSalary']==0, 'ProjSalary']=1
aa

,FG_PCT_ADJ,FT_PCT_ADJ,3PTM,PTS,REB,AST,ST,BLK,TO,Z_Sum,FirstName,LastName,ProjSalary,Name
4,-0.435012,3.049364,1.428815,2.192088,-0.580745,3.178772,0.172854,-1.082824,-2.725453,10.648765,Trae,Young,57,Trae Young
9,-0.857863,3.003426,1.120088,1.772792,1.101840,3.531897,1.093786,-0.105471,-3.024038,13.684533,James,Harden,49,James Harden
1,0.726554,2.667638,0.404401,2.043306,0.661871,1.125412,-0.556217,0.566460,-1.719692,9.359117,Kevin,Durant,60,Kevin Durant
15,-0.753490,2.599086,2.551460,2.205614,-0.432107,2.184789,-0.863194,-0.563605,-1.389677,8.318229,Damian,Lillard,41,Damian Lillard
20,0.053377,2.211578,-0.788409,1.899032,0.162446,1.053479,0.172854,-0.563605,-1.043947,5.244698,DeMar,DeRozan,33,DeMar DeRozan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0.806347,-2.588721,-1.433930,-0.497269,0.465668,0.988085,0.211226,-0.074929,-1.028232,-1.095290,Ben,Simmons,6,Ben Simmons
128,1.067118,-2.638066,-1.476029,-0.857954,2.290946,0.118350,0.172854,0.658086,-0.085331,-0.579363,Steven,Adams,1,Steven Adams
154,1.212375,-2.959977,-1.476029,-1.295285,0.501342,-0.234775,-0.594589,-0.013844,0.574699,-5.435482,Mason,Plumlee,1,Mason Plumlee
61,2.398713,-2.993520,-1.476029,-0.799343,1.553700,-1.169904,0.019365,2.857132,0.873285,-0.483171,Mitchell,Robinson,10,Mitchell Robinson


In [208]:
# list of zscores
# players = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
players = aa['Name'].values
zs = aa['Z_Sum'].values*-1
# zs = [9, 5, 10, 1, -1, 3, 5, 7, 12]
# zs = [i * -1 for i in zs]
#costs = [45, 25, 47, 4, 1, 14, 25, 35, 59]
aa.loc[aa['LastName']=='Jokic', 'ProjSalary'] = 80
costs = aa['ProjSalary'].values
zsd = dict(zip(players, zs))
costsd = dict(zip(players, costs))

In [241]:
prob= LpProblem("bball", LpMinimize)
player_vars = LpVariable.dicts("Players", players, lowBound=0, upBound=1, cat='Integer')

In [242]:
prob += lpSum([zsd[i]*player_vars[i] for i in players])


In [243]:
inv_map = {str(v): k for k, v in player_vars.items()}
inv_map

{'Players_Trae_Young': 'Trae Young',
 'Players_James_Harden': 'James Harden',
 'Players_Kevin_Durant': 'Kevin Durant',
 'Players_Damian_Lillard': 'Damian Lillard',
 'Players_DeMar_DeRozan': 'DeMar DeRozan',
 'Players_Stephen_Curry': 'Stephen Curry',
 'Players_Jimmy_Butler': 'Jimmy Butler',
 'Players_Kawhi_Leonard': 'Kawhi Leonard',
 'Players_Devin_Booker': 'Devin Booker',
 'Players_Darius_Garland': 'Darius Garland',
 'Players_Khris_Middleton': 'Khris Middleton',
 'Players_Bradley_Beal': 'Bradley Beal',
 'Players_Jayson_Tatum': 'Jayson Tatum',
 'Players_Kyrie_Irving': 'Kyrie Irving',
 'Players_Paul_George': 'Paul George',
 'Players_Kristaps_Porzingis': 'Kristaps Porzingis',
 'Players_Joel_Embiid': 'Joel Embiid',
 'Players_Karl_Anthony_Towns': 'Karl-Anthony Towns',
 'Players_Cade_Cunningham': 'Cade Cunningham',
 'Players_Immanuel_Quickley': 'Immanuel Quickley',
 'Players_Donovan_Mitchell': 'Donovan Mitchell',
 'Players_Fred_VanVleet': 'Fred VanVleet',
 'Players_Cole_Anthony': 'Cole Antho

In [244]:
prob += lpSum([costsd[i] * player_vars[i] for i in players]) <= 200.0
prob += lpSum([player_vars[i] for i in players]) == 13
prob += lpSum([player_vars[i] for i in banned_names]) == 0

In [245]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/23/9dnw19gx5276p1871tlf3q540000gn/T/c2ddec4197fd41fd85d50ad705ecd7d1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/23/9dnw19gx5276p1871tlf3q540000gn/T/c2ddec4197fd41fd85d50ad705ecd7d1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 1092 RHS
At line 1096 BOUNDS
At line 1311 ENDATA
Problem MODEL has 3 rows, 214 columns and 441 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -51.476 - 0.00 seconds
Cgl0002I 13 variables fixed
Cgl0004I processed model has 2 rows, 201 columns (201 integer (201 of which binary)) and 402 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.933333
Cbc0038I Pass   1: suminf.    0.28571 (2) obj. -50.8983 iterations 2
Cbc0

1

In [226]:
banned_names = [inv_map[b] for b in banned]

['Anthony Edwards',
 'Bam Adebayo',
 'Cade Cunningham',
 "De'Anthony Melton",
 'Devin Vassell',
 'Gary Trent Jr.',
 'JaVale McGee',
 'Jabari Smith II',
 "Jae'Sean Tate",
 'James Harden',
 'Kyle Lowry',
 'Marcus Smart',
 'Paul George']

In [246]:
for v in prob.variables():
    if v.varValue==1:
        print(v.name)

Players_Bennedict_Mathurin
Players_Bradley_Beal
Players_Buddy_Hield
Players_Cole_Anthony
Players_Domantas_Sabonis
Players_Dorian_Finney_Smith
Players_Harrison_Barnes
Players_Herbert_Jones
Players_Jalen_Suggs
Players_Khris_Middleton
Players_Luka_Doncic
Players_Monte_Morris
Players_Tyrese_Haliburton


In [194]:
banned

[Players_Anthony_Edwards,
 Players_Bam_Adebayo,
 Players_Cade_Cunningham,
 Players_De'Anthony_Melton,
 Players_Devin_Vassell,
 Players_Gary_Trent_Jr.,
 Players_JaVale_McGee,
 Players_Jabari_Smith_II,
 Players_Jae'Sean_Tate,
 Players_James_Harden,
 Players_Kyle_Lowry,
 Players_Marcus_Smart,
 Players_Paul_George]

In [155]:
aaa=[costsd['Bam Adebayo'], 
costsd['Devin Vassell'],
costsd['De\'Anthony Melton'], 
costsd['Dorian Finney-Smith'],
costsd['Harrison Barnes'], 
costsd['JaVale McGee'],
costsd['Jabari Smith II'], 
costsd['Jae\'Sean Tate'],
costsd['James Harden'], 
costsd['Kyle Lowry'],
costsd['Marcus Smart'], 
costsd['Nikola Jokic'],
costsd['Paul George']]

In [164]:
sum(aaa)

199

In [159]:
pd.set_option('display.max_rows', None)


In [248]:
# aa['ratio']=(aa['ProjSalary']/aa['Z_Sum']).mean()
# aa.sort_values('ratio', ascending=False)
(aa['ProjSalary']/aa['Z_Sum']).mean()

0.19073446619905912